In [6]:
# Keep Colab alive

import IPython
js_code = '''
function ClickConnect(){
console.log("Working");
  document.querySelector("colab-toolbar-button#connect").click();
  document.querySelector(".recaptcha-checkbox-border").click();
}
setInterval(ClickConnect,60000)
'''
display(IPython.display.Javascript(js_code))

<IPython.core.display.Javascript object>

In [7]:
!pip install keras-self-attention

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer 
from keras.utils import pad_sequences
from keras import backend as K 
import gensim
from numpy import *
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.utils import pad_sequences
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings

pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [9]:
from google.colab import drive
drive.mount('/content/drive')
%ls /content/drive/MyDrive/Datasets/

Mounted at /content/drive
xsum/                           xsum_40-150_to_10-30_val.csv
xsum_40-100_to_15-30_test.csv   xsum_40-250_to_8-30_test.csv
xsum_40-100_to_15-30_train.csv  xsum_40-250_to_8-30_train.csv
xsum_40-100_to_15-30_val.csv    xsum_40-250_to_8-30_val.csv
xsum_40-150_to_10-30_test.csv   xsum_reduced_prepared_150_50.csv
xsum_40-150_to_10-30_train.csv


In [10]:
# Sequences config:
SUM_MIN=10
SUM_MAX=30
TEXT_MIN=40
TEXT_MAX=150

In [11]:
# load ds by config
cl_test = pd.read_csv(f"/content/drive/MyDrive/Datasets/xsum_{TEXT_MIN}-{TEXT_MAX}_to_{SUM_MIN}-{SUM_MAX}_test.csv")
cl_val = pd.read_csv(f"/content/drive/MyDrive/Datasets/xsum_{TEXT_MIN}-{TEXT_MAX}_to_{SUM_MIN}-{SUM_MAX}_val.csv")
cl_train = pd.read_csv(f"/content/drive/MyDrive/Datasets/xsum_{TEXT_MIN}-{TEXT_MAX}_to_{SUM_MIN}-{SUM_MAX}_train.csv")

In [12]:
# no hyperparams yet, use merged Validation and Test
cl_valtest = pd.concat([cl_val, cl_test])
cl_valtest.reset_index(inplace=True, drop=True)

In [13]:
# select train and val set
# x_tr,x_val,y_tr,y_val=train_test_split(np.array(prepared['text']),np.array(prepared['summary']),test_size=0.1,random_state=0,shuffle=True)

# use directly split data:

x_tr = np.array(cl_train['text'])
y_tr = np.array(cl_train['summary'])

x_val = np.array(cl_valtest['text'])
y_val = np.array(cl_valtest['summary'])

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))

# rare word analysis - X
thresh=4
cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

#prepare a tokenizer for reviews on training data - X
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences (i.e one-hot encodeing all the words)
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen=TEXT_MAX, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=TEXT_MAX, padding='post')

#size of vocabulary ( +1 for padding token)
x_voc   =  x_tokenizer.num_words + 1

print("Size of vocabulary in X = {}".format(x_voc))

#prepare a tokenizer for reviews on training data - Y
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

# rare word analysis - Y
thresh=6
cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
y_tokenizer.fit_on_texts(list(y_tr))

#convert text sequences into integer sequences (i.e one hot encode the text in Y)
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen=SUM_MAX, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=SUM_MAX, padding='post')

#size of vocabulary
y_voc  =   y_tokenizer.num_words +1
print("Size of vocabulary in Y = {}".format(y_voc))

# remove empty summaries
ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)
ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

print("Size of vocabulary from the w2v model = {}".format(x_voc))


% of rare words in vocabulary: 63.16852282462367
Total Coverage of rare words: 1.9231407644074598
Size of vocabulary in X = 30096
% of rare words in vocabulary: 73.86055519690122
Total Coverage of rare words: 4.5112503647060365
Size of vocabulary in Y = 8099
Size of vocabulary from the w2v model = 30096


In [14]:
pd.DataFrame(x_tr)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,1,266,22895,4,16480,2445,14650,6186,241,3,...,0,0,0,0,0,0,0,0,0,0
1,14,7,1851,3312,5,1,2624,967,3103,3028,...,0,0,0,0,0,0,0,0,0,0
2,4,3609,777,5439,16481,1158,2,1,549,5298,...,260,3,1534,64,29,10213,12,97,2,1015
3,10214,10,38,29,514,6952,5,1663,45,1,...,0,0,0,0,0,0,0,0,0,0
4,1,2859,17,35,10532,521,7186,4780,415,29,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40108,1,7109,2634,95,15,2517,98,227,3,179,...,0,0,0,0,0,0,0,0,0,0
40109,3726,209,193,33,5928,594,8,1,80,1098,...,0,0,0,0,0,0,0,0,0,0
40110,4248,7,116,583,31,1234,111,131,31,133,...,0,0,0,0,0,0,0,0,0,0
40111,1,82,1633,1995,30,355,61,6,623,177,...,0,0,0,0,0,0,0,0,0,0


In [15]:
pd.DataFrame(y_tr)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,3,2,30,820,30,493,7305,160,108,5,...,4,0,0,0,0,0,0,0,0,0
1,2,6,30,5,131,733,25,1362,9,1469,...,61,17,5,3535,7,5,2662,6729,1,4
2,3,2,3232,8,4109,16,23,1003,1604,2367,...,0,0,0,0,0,0,0,0,0,0
3,3,2,2862,114,161,821,2949,38,41,4680,...,2495,5866,1,4,0,0,0,0,0,0
4,3,2,1363,8,973,234,7,227,13,5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40108,3,2,555,2240,5439,4636,9,203,49,167,...,0,0,0,0,0,0,0,0,0,0
40109,3,2,49,4944,217,3524,125,57,8,2236,...,1,4,0,0,0,0,0,0,0,0
40110,3,2,818,103,2543,3339,8,11,47,18,...,0,0,0,0,0,0,0,0,0,0
40111,3,2,161,415,5448,1114,1955,14,3929,27,...,5,1,4,0,0,0,0,0,0,0


In [16]:
# model setup

K.clear_session()

# Parameters
latent_dim = 700 # orig 300
embedding_dim = 200 # orig 200

# Encoder
encoder_inputs = Input(shape=(TEXT_MAX,))

#embedding layer
enc_emb = Embedding(x_voc, embedding_dim,trainable=True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

print(model.summary())
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy'
)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 150, 200)     6019200     ['input_1[0][0]']                
                                                                                                  
 lstm (LSTM)                    [(None, 150, 700),   2522800     ['embedding[0][0]']              
                                 (None, 700),                                                     
                                 (None, 700)]                                                     
                                                                                              

In [19]:
# train setup
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)


modeldir = f"/content/drive/MyDrive/idabssum_latent({latent_dim})_embedding({embedding_dim})_xsum_{TEXT_MIN}-{TEXT_MAX}_to_{SUM_MIN}-{SUM_MAX}"
checkpoint_filepath = modeldir + '/checkpoint.hdf'
print('Target path:', checkpoint_filepath)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

csv_logger=tf.keras.callbacks.CSVLogger(modeldir + '/log.csv', separator=",", append=True)

try:
  model.load_weights(checkpoint_filepath)
  print('Loaded model weights checkpoint.')
except:
  print('Cannot load model weights from checkpoint, it may not exist yet.')

Target path: /content/drive/MyDrive/idabssum_latent(700)_embedding(200)_xsum_40-150_to_10-30/checkpoint.hdf
Loaded model weights checkpoint.


In [18]:
# training
do_train = True

if do_train:
  history = model.fit(
      [x_tr, y_tr[:, :-1]],
      y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)[:, 1:],
      epochs=50,
      callbacks=[es, model_checkpoint_callback, csv_logger],
      batch_size=64,
      validation_data=(
          [x_val, y_val[:, :-1]],
          y_val.reshape(y_val.shape[0], y_val.shape[1], 1)[:, 1:],
      ),
  )
  # training visualisation
  from matplotlib import pyplot
  pyplot.plot(history.history['loss'], label='train')
  pyplot.plot(history.history['val_loss'], label='test')
  pyplot.legend()
  pyplot.show()

Epoch 1/50
627/627 [==============================] - 1535s 2s/step - loss: 2.6626 - val_loss: 2.7014
Epoch 2/50
627/627 [==============================] - 1489s 2s/step - loss: 2.5466 - val_loss: 2.6515
Epoch 3/50
627/627 [==============================] - 1470s 2s/step - loss: 2.4291 - val_loss: 2.6143
Epoch 4/50
627/627 [==============================] - 1478s 2s/step - loss: 2.3389 - val_loss: 2.5959
Epoch 5/50
627/627 [==============================] - 1483s 2s/step - loss: 2.2675 - val_loss: 2.5858
Epoch 6/50
  2/627 [..............................] - ETA: 23:43 - loss: 2.0484

KeyboardInterrupt: ignored

In [20]:
# target to source dictionary setup and decoder

reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(TEXT_MAX,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c], verbose=0)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (SUM_MAX-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

# finally apply model
for i in range(0,10):
    print("Review:",seq2text(x_tr[i]))
    print("Original summary:",seq2summary(y_tr[i]))
    print("Predicted summary:",decode_sequence(x_tr[i].reshape(1,TEXT_MAX)))
    print("\n")

Review: the national oceanic and atmospheric administration noaa forecasts 14 to 23 named storms eight to 14 of these could develop into hurricanes it says the season could be one of the most active on record with between three to seven major hurricanes the noaa said in short we urge everyone to be prepared added spokesperson for the agency said major hurricanes can reach category three or higher which means they bring sustained winds of at least h the noaa says the seasonal forecasts have 70 the hurricane season begins on june and runs until the end of november 
Original summary: start as many as 14 hurricanes could hit the atlantic this year the top us climate agency says end 
1/1 [==============================] - 2s 2s/step
Predicted summary:  start a man has been arrested on suspicion of murder after man was stabbed to death in north london end


Review: it was rare example of the cold war turning hot pitting the us and its allies against the north korea and communist china it was

KeyboardInterrupt: ignored

In [ ]:
sekv = x_tokenizer.texts_to_sequences(['sostok _START_ a memorial has been held for killing a woman in car crash of two lorries _END_ eostok'])
padded = pad_sequences(sekv, maxlen=TEXT_MAX, padding='post')
print(seq2text(padded[0])) # ok base sequence

decode_sequence(padded[0].reshape(1,TEXT_MAX))